In [7]:
import numpy as np
from scipy.signal import butter, lfilter, freqz
import matplotlib.pyplot as plt
import pandas as pd

In [3]:


def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y



In [9]:
features = pd.DataFrame.from_csv("../../data/global_acc_features_df.csv")


In [36]:
groups = features.groupby(['TagName','tester_id'])

In [41]:
features.head()

,unix_timestamp,TagName,tester_id,v_1,v_2,v_3,d_1,d_2,d_3,global_acc1,global_acc2,global_acc3
0,1.515398e+09,Tag0,0.0,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,-0.058561,-0.130082,-0.520824
1,1.515398e+09,Tag0,0.0,-0.000073,-0.000388,-0.002020,-1.834992e-07,-9.703653e-07,-0.000005,0.021859,-0.064001,-0.489335
2,1.515398e+09,Tag0,0.0,0.000035,-0.000564,-0.004001,-4.303650e-07,-3.528084e-06,-0.000020,-0.002340,-0.037200,-0.522459
3,1.515398e+09,Tag0,0.0,-0.000160,-0.000652,-0.006500,-5.877165e-07,-7.142123e-06,-0.000052,-0.070036,0.008021,-0.501289
4,1.515398e+09,Tag0,0.0,-0.000465,-0.000570,-0.008448,-2.219124e-06,-1.039287e-05,-0.000089,-0.069951,0.008737,-0.501289


In [35]:
def lowpassfilter(data):
    fc = 0.1
    b = 0.08
    N = int(np.ceil((4 / b)))
    if not N % 2: N += 1
    n = np.arange(N)

    sinc_func = np.sinc(2 * fc * (n - (N - 1) / 2.))
    window = 0.42 - 0.5 * np.cos(2 * np.pi * n / (N - 1)) + 0.08 * np.cos(4 * np.pi * n / (N - 1))
    sinc_func = sinc_func * window
    sinc_func = sinc_func / np.sum(sinc_func)

    new_signal = np.convolve(data, sinc_func)
    return new_signal

In [48]:
filterred_feature = pd.DataFrame(columns=['TagName','tester_id','filtered_global_acc_1',
                                          'filtered_global_acc_2','filtered_global_acc_3'])
for key in groups.groups.keys():
    
    cur_group = groups.get_group(key)
    temp = pd.DataFrame(columns=['TagName','tester_id','filtered_global_acc_1','filtered_global_acc_2',
                                 'filtered_global_acc_3'])
    temp['filtered_global_acc_1'] = lowpassfilter(cur_group.global_acc1.values)
    temp['filtered_global_acc_2'] = lowpassfilter(cur_group.global_acc2.values)
    temp['filtered_global_acc_3'] = lowpassfilter(cur_group.global_acc3.values)
    temp['TagName'] = key[0]
    temp['tester_id'] = key[1]
    filterred_feature = filterred_feature.append(temp, ignore_index=True)
filterred_feature.to_csv("../../data/filterred_feature.csv")